In [1]:
#  Ce bloc permet de télécharger le dataset Tiny LISA depuis Kaggle.
# L'utilisateur doit d'abord importer son propre fichier kaggle.json (clé API personnelle)
import os

# Création du dossier Kaggle
os.makedirs('/root/.kaggle', exist_ok=True)

# Télécharger kaggle.json depuis votre machine locale (Colab uniquement)
from google.colab import files
uploaded = files.upload()

# Déplacement de kaggle.json vers ~/.kaggle/
os.rename("kaggle.json", "/root/.kaggle/kaggle.json")

# Changer les permissions
os.chmod("/root/.kaggle/kaggle.json", 0o600)

# Installation de la bibliothèque Kaggle
!pip install kaggle --quiet

# 2. Télécharger et extraire le dataset
!kaggle datasets download -d mmontiel/tiny-lisa-traffic-sign-detection-dataset
!unzip -q tiny-lisa-traffic-sign-detection-dataset.zip -d data


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/mmontiel/tiny-lisa-traffic-sign-detection-dataset
License(s): unknown
 83% 244M/292M [00:00<00:00, 496MB/s]
100% 292M/292M [00:00<00:00, 535MB/s]


In [2]:
!mkdir /content/data/db_lisa_tiny/images
!mkdir /content/data/db_lisa_tiny/annotations


In [3]:
!mv /content/data/db_lisa_tiny/sample_*.png /content/data/db_lisa_tiny/images/
!mv /content/data/db_lisa_tiny/annotations.csv /content/data/db_lisa_tiny/annotations/


Yolov5


In [4]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17496, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17496 (delta 2), reused 0 (delta 0), pack-reused 17491 (from 3)
Receiving objects: 100% (17496/17496), 16.59 MiB | 16.62 MiB/s, done.
Resolving deltas: 100% (11993/11993), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:0

In [5]:
import pandas as pd
df = pd.read_csv("/content/data/db_lisa_tiny/annotations/annotations.csv")
df.head()

,filename,x1,y1,x2,y2,class
0,sample_001.png,190,40,211,63,stop
1,sample_002.png,4,246,43,283,stop
2,sample_003.png,389,286,418,314,stop
3,sample_004.png,307,243,315,251,stop
4,sample_005.png,377,249,398,270,stop


In [6]:
import pandas as pd
import os

# Charger les annotations
df = pd.read_csv('/content/data/db_lisa_tiny/annotations/annotations.csv', sep=',')

# Liste des classes (YOLO a besoin d'un ID numérique)
classes = ['stop' , 'yield' , 'yieldAhead' , 'merge', 'signalAhead' , 'pedestrianCrossing', 'keepRight' ,'speedLimit35' , 'speedLimit25']  # Ajoute d'autres classes si nécessaire

# Dossier de sortie
output_dir = '/content/data/db_lisa_tiny/labels/'
os.makedirs(output_dir, exist_ok=True)

# Traiter chaque ligne du CSV
for _, row in df.iterrows():
    filename = row['filename']
    x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
    class_name = row['class']

    # Charger l'image pour obtenir ses dimensions
    img_path = f"/content/data/db_lisa_tiny/images/{filename}"
    try:
        from PIL import Image
        img = Image.open(img_path)
        img_width, img_height = img.size
    except:
        print(f"Impossible de charger l'image {filename}, vérifiez son emplacement.")
        continue

    # Convertir en coordonnées YOLO
    x_center = (x1 + x2) / 2 / img_width
    y_center = (y1 + y2) / 2 / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height

    # ID de la classe
    class_id = classes.index(class_name)

    # Sauvegarde dans un fichier texte
    label_filename = os.path.join(output_dir, filename.replace('.png', '.txt'))
    with open(label_filename, 'w') as f:
        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

print("Conversion terminée ! Les fichiers YOLO sont dans :", output_dir)


✅ Conversion terminée ! Les fichiers YOLO sont dans : /content/data/db_lisa_tiny/labels/


In [7]:
!mkdir -p /content/data/db_lisa_tiny/images/train
!mkdir -p /content/data/db_lisa_tiny/images/val
!mkdir -p /content/data/db_lisa_tiny/labels/train
!mkdir -p /content/data/db_lisa_tiny/labels/val


In [8]:
import os
import shutil
import random
from glob import glob

# Définir les chemins
image_dir = "/content/data/db_lisa_tiny/images"
label_dir = "/content/data/db_lisa_tiny/labels"

train_img_dir = "/content/data/db_lisa_tiny/images/train"
val_img_dir = "/content/data/db_lisa_tiny/images/val"
train_label_dir = "/content/data/db_lisa_tiny/labels/train"
val_label_dir = "/content/data/db_lisa_tiny/labels/val"

# Lister toutes les images
image_files = glob(os.path.join(image_dir, "*.png"))  # Change en *.jpg si besoin

# Mélanger les images pour une répartition aléatoire
random.shuffle(image_files)

# Séparer en 80% train, 20% val
split_index = int(len(image_files) * 0.8)
train_files = image_files[:split_index]
val_files = image_files[split_index:]

# Fonction pour déplacer les fichiers
def move_files(file_list, dest_img_dir, dest_label_dir):
    for img_path in file_list:
        filename = os.path.basename(img_path)
        label_filename = filename.replace(".png",".txt")  # Adapter selon l'extension

        # Déplacer l'image
        shutil.move(img_path, os.path.join(dest_img_dir, filename))

        # Déplacer l'annotation correspondante
        label_path = os.path.join(label_dir, label_filename)
        if os.path.exists(label_path):
            shutil.move(label_path, os.path.join(dest_label_dir, label_filename))

# Appliquer la séparation
move_files(train_files, train_img_dir, train_label_dir)
move_files(val_files, val_img_dir, val_label_dir)

print(" Séparation terminée !")
print(f"Train : {len(train_files)} images")
print(f"Val : {len(val_files)} images")


✅ Séparation terminée !
Train : 720 images
Val : 180 images


In [9]:
yaml_content = """train: /content/data/db_lisa_tiny/images/train
val: /content/data/db_lisa_tiny/images/val

nc: 9
names: ['stop', 'yield', 'yieldAhead', 'merge', 'signalAhead', 'pedestrianCrossing', 'keepRight', 'speedLimit35', 'speedLimit25']
"""

with open('/content/data/lisa.yaml', 'w') as f:
    f.write(yaml_content)

print(" Fichier lisa.yaml créé avec succès !")


✅ Fichier lisa.yaml créé avec succès !


In [10]:
!pip install torch torchvision torchaudio --quiet


In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data /content/data/lisa.yaml --weights yolov5s.pt --cache


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-06-28 14:46:19.668410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751121979.689476    1332 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751121979.695997    1332 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source /content/data/db_lisa_tiny/images/val
